In [2]:


import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

data={
    'user':['user1','user2','user3','user4'],
    'item1':[5,4,3,2],
        'item2':[4,5,2,3],
        'item3':[2,3,5,4]
}

df=pd.DataFrame(data).set_index('user')

user_similarity=cosine_similarity(df);
print("using cosine functions: ",user_similarity);

pearson=df.T.corr(method='pearson')

print("pcc_similarity",pearson);


using cosine functions:  [[1.         0.96976515 0.79802388 0.8304548 ]
 [0.96976515 1.         0.84883822 0.91914503]
 [0.79802388 0.84883822 1.         0.96396037]
 [0.8304548  0.91914503 0.96396037 1.        ]]
pcc_similarity user      user1     user2     user3     user4
user                                         
user1  1.000000  0.654654 -0.785714 -0.981981
user2  0.654654  1.000000 -0.981981 -0.500000
user3 -0.785714 -0.981981  1.000000  0.654654
user4 -0.981981 -0.500000  0.654654  1.000000


In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import  cosine_similarity

data={
    'user':['user1','user2','user3','user4','user5'],
    'item1':[5,4,3,2,1],
    'item2':[4,5,2,3,6],  
    'item3':[2,3,5,4,5],
    'item4':[4,3,3,5,2],
    'item5':[np.nan,3,5,4,1],
}

df=pd.DataFrame(data).set_index("user")
user_similarity=df.T.corr(method="pearson")
print(user_similarity)

    

user      user1     user2     user3     user4     user5
user                                                   
user1  1.000000  0.484200 -0.789474 -0.512989 -0.612056
user2  0.484200  1.000000 -0.791667 -0.686406  0.476731
user3 -0.789474 -0.791667  1.000000  0.359546 -0.238366
user4 -0.512989 -0.686406  0.359546  1.000000  0.000000
user5 -0.612056  0.476731 -0.238366  0.000000  1.000000


In [11]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

data = {
    'User': ['User1', 'User2', 'User3', 'User4', 'user5'],
    'Item1': [5, 3, 4, 3, 1],
    'Item2': [3, 1, 3, 3, 5],
    'Item3': [4, 2, 4, 1, 5],
    'Item4': [4, 3, 3, 5, 2],
    'Item5': [np.nan, 3, 5, 4, 1],
}


df = pd.DataFrame(data).set_index('User')
user_similarity = df.T.corr(method='pearson')  # Pearson correlation
print(user_similarity)

user_similarity_df = pd.DataFrame(user_similarity, index=df.index, columns=df.index)

def predict_rating(user, item):
    # Filter out NaN values from the user similarity and item ratings
    similarity_scores = user_similarity_df[user]
    item_ratings = df[item]

    # Only consider non-NaN ratings
    mask = ~item_ratings.isna()
    similarity_scores = similarity_scores[mask]
    item_ratings = item_ratings[mask]

    if similarity_scores.empty or item_ratings.empty:
        return 0  # Return 0 if no relevant data for prediction

    numerator = np.sum(similarity_scores * item_ratings)
    denominator = np.sum(np.abs(similarity_scores))

    if denominator == 0:
        return 0
    return numerator / denominator

def recommend_items(user):
    unrated_items = df.columns[pd.isna(df.loc[user])]  # Items that the user hasn't rated
    predictions = [predict_rating(user, item) for item in unrated_items]
    recommendations = pd.DataFrame({'Item': unrated_items, 'Prediction': predictions})
    return recommendations.sort_values(by='Prediction', ascending=False)

user_to_recommend = 'User1'
recommendations = recommend_items(user_to_recommend)
print(f"Recommendations for {user_to_recommend}:")
print(recommendations)

User      User1     User2     User3     User4     user5
User                                                   
User1  1.000000  0.852803  0.707107  0.000000 -0.792118
User2  0.852803  1.000000  0.467707  0.489956 -0.900149
User3  0.707107  0.467707  1.000000 -0.161165 -0.466569
User4  0.000000  0.489956 -0.161165  1.000000 -0.641503
user5 -0.792118 -0.900149 -0.466569 -0.641503  1.000000
Recommendations for User1:
    Item  Prediction
0  Item5    2.254151


In [2]:
n #WITH MEAN CENTRIC COSINE

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Define the data
data = {
    'User': ['Alice', 'User1', 'User2', 'User3', 'User4'],
    'Item1': [5, 3, 4, 3, 1],
    'Item2': [3, 1, 3, 3, 5],
    'Item3': [4, 2, 4, 1, 5],
    'Item4': [4, 3, 3, 5, 2],
    'Item5': [np.nan, 3, 5, 4, 1],
}

# Create the DataFrame
df = pd.DataFrame(data).set_index('User')

# Calculate the mean rating for each user, excluding NaN values
user_means = df.mean(axis=1)

# Mean-center the ratings (subtract each user's mean from their ratings)
mean_centered_df = df.sub(user_means, axis=0)

# Replace NaN values with 0 for cosine similarity calculation
cosine_input = mean_centered_df.fillna(0)

# Compute user-user cosine similarity
cosine_sim_matrix = cosine_similarity(cosine_input)
user_similarity_df = pd.DataFrame(cosine_sim_matrix, index=df.index, columns=df.index)

print("\nUser-User Similarity (Mean-Centered, Cosine):")
print(user_similarity_df)

# Function to predict missing values using user-based collaborative filtering
def predict_missing_value(df, user_similarity_df, target_user, target_item, user_means):
    # Check if the target item is actually missing for the target user
    if not np.isnan(df.loc[target_user, target_item]):
        return df.loc[target_user, target_item]
    
    # Initialize numerator and denominator
    numerator = 0
    denominator = 0
    
    # Iterate over all users
    for other_user in df.index:
        if other_user == target_user or np.isnan(df.loc[other_user, target_item]):
            continue
        
        # Get similarity and rating of the other user
        similarity = user_similarity_df.loc[target_user, other_user]
        if similarity > 0:  # Only consider positive similarities
            rating = df.loc[other_user, target_item]
            
            # Accumulate weighted ratings (mean-centered) and similarities
            numerator += similarity * (rating - user_means[other_user])
            denominator += abs(similarity)
    
    # Predict the value by adding back the target user's mean
    return user_means[target_user] + (numerator / denominator if denominator != 0 else 0)

# Predict the missing value for Alice's Item5
predicted_value = predict_missing_value(df, user_similarity_df, target_user='Alice', target_item='Item5', user_means=user_means)

# Update the DataFrame with the predicted value
df.loc['Alice', 'Item5'] = predicted_value

# Display updated DataFrame
print("\nUpdated DataFrame with Missing Value Filled:")
print(df)


User-User Similarity (Mean-Centered, Cosine):
User      Alice     User1     User2     User3     User4
User                                                   
Alice  1.000000  0.790569  0.422577  0.000000 -0.690066
User1  0.790569  1.000000  0.467707  0.489956 -0.900149
User2  0.422577  0.467707  1.000000 -0.161165 -0.466569
User3  0.000000  0.489956 -0.161165  1.000000 -0.641503
User4 -0.690066 -0.900149 -0.466569 -0.641503  1.000000

Updated DataFrame with Missing Value Filled:
       Item1  Item2  Item3  Item4     Item5
User                                       
Alice      5      3      4      4  4.808999
User1      3      1      2      3  3.000000
User2      4      3      4      3  5.000000
User3      3      3      1      5  4.000000
User4      1      5      5      2  1.000000


In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

data = {
    'User': ['Alice', 'User1', 'User2', 'User3', 'User4'],
    'Item1': [5, 3, 4, 3, 1],
    'Item2': [3, 1, 3, 3, 5],
    'Item3': [4, 2, 4, 1, 5],
    'Item4': [4, 3, 3, 5, 2],
    'Item5': [np.nan, 3, 5, 4, 1],
}
# Create the DataFrame
df = pd.DataFrame(data).set_index('User')
cosine_input = df.fillna(0)
# Compute user-user similarity using Pearson Correlation
cosine_sim_matrix = cosine_similarity(cosine_input)
user_similarity_df = pd.DataFrame(cosine_sim_matrix, index=df.index, columns=df.index)

print("\nSimilarities of the dataframe")
print(user_similarity_df)


# Function to predict missing values using user-based collaborative filtering
def predict_missing_value(df, user_similarity_df, target_user, target_item):
    # Check if the target item is actually missing for the target user
    if not np.isnan(df.loc[target_user, target_item]):
        return df.loc[target_user, target_item]
    
    # Initialize numerator and denominator
    numerator = 0
    denominator = 0
    
    # Iterate over all users
    for other_user in df.index:
        if other_user == target_user or np.isnan(df.loc[other_user, target_item]):
            continue
        
        # Get similarity and rating of the other user
        if(user_similarity_df.loc[target_user, other_user]>0):
            similarity = user_similarity_df.loc[target_user, other_user]
            rating = df.loc[other_user, target_item]
            
            # Accumulate weighted ratings and similarities
            numerator += similarity * rating
            denominator += abs(similarity)
        
        
    # Calculate and return predicted value
    return numerator / denominator if denominator != 0 else np.nan

# Predict the missing value for Alice's Item5
predicted_value = predict_missing_value(df, user_similarity_df, target_user='Alice', target_item='Item5')

# Update the DataFrame with the predicted value
df.loc['Alice', 'Item5'] = predicted_value

# Display updated DataFrame
print("\nUpdated DataFrame with Missing Value Filled:")
print(df)


Similarities of the dataframe
User      Alice     User1     User2     User3     User4
User                                                   
Alice  1.000000  0.826869  0.810163  0.762770  0.789542
User1  0.826869  1.000000  0.959383  0.935693  0.637815
User2  0.810163  0.959383  1.000000  0.894427  0.771517
User3  0.762770  0.935693  0.894427  1.000000  0.638311
User4  0.789542  0.637815  0.771517  0.638311  1.000000

Updated DataFrame with Missing Value Filled:
       Item1  Item2  Item3  Item4     Item5
User                                       
Alice      5      3      4      4  3.252093
User1      3      1      2      3  3.000000
User2      4      3      4      3  5.000000
User3      3      3      1      5  4.000000
User4      1      5      5      2  1.000000


In [1]:
#MATRIX FACTORIZATION
import numpy
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.01, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j]) 
                    for k in range(K): 
                        P[i][k] = P[i][k] + alpha * (eij * Q[k][j] - beta * P[i][k]) 
                        Q[k][j] = Q[k][j] + alpha * (eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot (P,Q)
    return P, Q.T
R = [[5,3,0,1],
     [4,0,0,1],
     [1,1,0,5],
     [0,3,4,0], ]
R = numpy.array(R)
N = len(R)
M = len(R[0])
K = 2
# P = numpy.random.rand(N,K)
# Q = numpy.random.rand(M,K)
P=[[0.5,0.8],
   [0.3,0.6],
   [0.9,0.4],
   [0.2,0.7], ]
Q=[[0.6,0.2],
   [0.1,0.5],
   [0.8,0.9],
   [0.3,0.4], ]
P = numpy.array(P)
Q = numpy.array(Q)
nP, nQ = matrix_factorization(R, P, Q, K)
nR = numpy.dot(nP, nQ.T)
print('nr',nR)

  

nr [[4.95348425 2.98879899 3.97631571 1.00014954]
 [3.97144631 2.41264473 3.2194069  0.99817691]
 [1.00122875 0.99979729 1.5622703  4.94399989]
 [4.83309146 2.9690518  3.98107545 1.60972726]]


In [3]:
import numpy as np
from numpy.linalg import svd

def svd_impute(data, k=2):
    data = np.array(data, dtype=np.float64)
    mask = data == 0  # Identify missing values
    mean_values = np.mean(data, axis=1, where=~mask, keepdims=True)
    # Tile mean_values to match the shape of mask
    mean_values = np.tile(mean_values, (1, data.shape[1]))
    data[mask] = mean_values[mask]  # Temporary fill with row mean

    U, S, Vt = svd(data, full_matrices=False)  # Compute SVD
    S = np.diag(S[:k])  # Reduce rank to k
    U = U[:, :k]
    Vt = Vt[:k, :]
    print('U',U)
    print('S',S)
    print('Vt',Vt)

    reconstructed = U @ S @ Vt  # Reconstruct the matrix

    data[mask] = reconstructed[mask]  # Replace only missing values
    return data

data = [
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [0, 3, 4, 0],
]

predicted_ratings = svd_impute(data, k=2)
print("\nmissing predicted")

print(np.round(predicted_ratings, 2))


U [[-0.54160611 -0.46810733]
 [-0.44886273 -0.34079387]
 [-0.37893089  0.79651311]
 [-0.6013289   0.17407452]]
S [[11.595673    0.        ]
 [ 0.          4.53422491]]
Vt [[-0.60255783 -0.42514934 -0.52057891 -0.43031348]
 [-0.50679847 -0.20677625  0.06583749  0.8343047 ]]

missing predicted
[[5.   3.   3.13 1.  ]
 [4.   2.53 2.61 1.  ]
 [1.   1.   2.53 5.  ]
 [3.8  3.   4.   3.66]]


In [4]:
import numpy as np
from sklearn.decomposition import PCA

# Given data matrix
data = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [0, 3, 4, 0],
], dtype=float)

# Mask to identify missing values (0 represents missing values)
mask = data > 0

# Fill missing values with the mean of the corresponding column
column_means = np.nanmean(np.where(mask, data, np.nan), axis=0)
filled_data = np.where(mask, data, column_means)

# Apply PCA for dimensionality reduction
n_components = 2  # Number of principal components (adjust as needed)
pca = PCA(n_components=n_components)
reduced_data = pca.fit_transform(filled_data)

# Reconstruct the matrix from the reduced data
reconstructed_data = pca.inverse_transform(reduced_data)

# Replace missing values with predicted values
predicted_data = data.copy()
predicted_data[~mask] = reconstructed_data[~mask]

print("Original Data:")
print(data)
print("\nPredicted Data:")
print(predicted_data)

Original Data:
[[5. 3. 0. 1.]
 [4. 0. 0. 1.]
 [1. 1. 0. 5.]
 [0. 3. 4. 0.]]

Predicted Data:
[[5.         3.         4.         1.        ]
 [4.         2.26192246 4.         1.        ]
 [1.         1.         4.         5.        ]
 [3.51085025 3.         4.         2.45779723]]
